In [1]:
#安裝跑 PySpark 的環境
!apt-get -y install openjdk-8-jre-headless
!pip install -q pyspark

#install Apache Spark 3.1.2 with Hadoop 2.7 from here.
!wget -q https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
#we just need to unzip that folder.
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
# findspark library will locate Spark on the system and import it as a regular library.

!pip install -q findspark
#set the environment path. This will enable us to run Pyspark in the Colab environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"


#We need to locate Spark in the system. For that, we import findspark and use the findspark.init() method.

import findspark
findspark.init()
#know the location where Spark is installed, use findspark.find()
findspark.find()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jre-headless
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 28.2 MB of archives.
After this operation, 104 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u292-b10-0ubuntu1~18.04 [28.2 MB]
Fetched 28.2 MB in 5s (6,257 kB/s)
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155062 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u292-b10-0ubuntu1~18.04_amd64

'/content/spark-3.1.2-bin-hadoop2.7'

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# yarn mode
#spark = SparkSession\
#        .builder\
#        .master("yarn")\
#        .config('spark.executor.instances','99')\
#        .config('spark.executor.memory','4G')\
#        .appName("trainRF")\
#        .getOrCreate()

In [3]:
# Check spark app name
spark.sparkContext.appName

'Colab'

In [4]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)

In [5]:
!pip install databricks
!pip install koalas

     |████████████████████████████████| 390 kB 4.3 MB/s 


In [6]:
# need export ARROW_PRE_0_15_IPC_FORMAT=1 in .bashrc
import databricks.koalas as ks

In [7]:
ks.set_option("compute.default_index_type", "distributed")

In [8]:
from pyspark import SparkContext, SparkConf
sc = SparkContext.getOrCreate()
# 使用Word Count範例程式，驗證PySpark環境是否可以正常運作
rdd = sc.parallelize(["Hello Spark"])
counts = rdd.flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()
print(counts)

[('Hello', 1), ('Spark', 1)]


In [9]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 1.8.0_292
Branch HEAD
Compiled by user centos on 2021-05-24T04:46:13Z
Revision de351e30a90dd988b133b3d00fa6218bfcaba8b8
Url https://github.com/apache/spark
Type --help for more information.


In [10]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-11-01 13:04:44--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  5.55MB/s    in 2.4s    

2021-11-01 13:04:47 (5.55 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
{"tunnels":[],"uri":"/api/tunnels"}


In [11]:
import pandas as pd
from pandas import read_csv
import numpy
from numpy import allclose
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorAssembler

In [12]:
# 3.2才可用
#import pyspark.pandas as pd
#from pyspark.pandas import read_csv
# 沒用到pyspark
import sklearn 
from xgboost import XGBClassifier #scikit learn API https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# from sklearn.feature_selection import SequentialFeatureSelector 會報錯 不支援sklearn 0.24
from sklearn.model_selection import train_test_split

In [13]:
df = spark.read.csv('/content/drive/MyDrive/project/data/all_features_train_OneHot.csv', header=True, inferSchema=True)
df.createOrReplaceTempView("dfTable")  # 為了可執行sql command
df.show(5)

+---+-----+-----------+--------+--------------------+---------------------+------------------+---------------------------------+-------------------------+------------------------+-----------+---------------+---------------+------------------+--------------------+--------------------+---------------------+-----------------------+-----------------------+---------------------+-----------------------+-----------------------+---------------------+-----------------------+-----------------------+----------------------+------------------------+------------------------+-------------------+---------------------+---------------------+----------------------+------------------------+------------------------+----------------------+------------------------+------------------------+----------------------+------------------------+------------------------+-----------------------+-------------------------+-------------------------+----------------+------------------+------------------+-------------------

In [14]:
df.count(), len(df.columns)

(160057, 312)

In [15]:
df.groupBy('label').count().show()

+-----+------+
|label| count|
+-----+------+
|    1| 43438|
|    0|116619|
+-----+------+



In [16]:
feature_cols = df.columns[4:]
print(feature_cols)
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
assembler
df = assembler.transform(df)

['never_bought_company', 'never_bought_category', 'never_bought_brand', 'has_bought_brand_company_category', 'has_bought_brand_category', 'has_bought_brand_company', 'offer_value', 'total_spend_all', 'total_spend_ccb', 'has_bought_company', 'has_bought_company_q', 'has_bought_company_a', 'has_bought_company_30', 'has_bought_company_q_30', 'has_bought_company_a_30', 'has_bought_company_60', 'has_bought_company_q_60', 'has_bought_company_a_60', 'has_bought_company_90', 'has_bought_company_q_90', 'has_bought_company_a_90', 'has_bought_company_180', 'has_bought_company_q_180', 'has_bought_company_a_180', 'has_bought_category', 'has_bought_category_q', 'has_bought_category_a', 'has_bought_category_30', 'has_bought_category_q_30', 'has_bought_category_a_30', 'has_bought_category_60', 'has_bought_category_q_60', 'has_bought_category_a_60', 'has_bought_category_90', 'has_bought_category_q_90', 'has_bought_category_a_90', 'has_bought_category_180', 'has_bought_category_q_180', 'has_bought_categ

In [3]:
data = df.select(['features', 'label'])
data.show(10)

NameError: ignored

In [18]:
data.select(['label']).distinct().show()

+-----+
|label|
+-----+
|    1|
|    0|
+-----+



In [19]:
X_train, X_test = data.randomSplit([0.80, 0.20], seed=25)

X_train.count(), len(X_train.columns)

(128249, 2)

In [20]:
X_test.count(), len(X_test.columns)

(31808, 2)

In [21]:
%%timeit
X_test.count(), len(X_test.columns)

1 loop, best of 5: 8.1 s per loop


In [22]:
import sys
import time
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.regression import RandomForestRegressor

In [24]:
rf = RandomForestClassifier(numTrees=100, maxDepth=15, labelCol="label", featuresCol='features', seed=42)
model = rf.fit(X_train)

In [ ]:
%%timeit
rf = RandomForestClassifier(numTrees=100, maxDepth=15, labelCol="label", featuresCol='features', seed=42)
model = rf.fit(X_train)

In [25]:
prediction = model.transform(X_test)
prediction.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(308,[0,1,2,6,7,8...|    1|[52.6255588184475...|[0.52625558818447...|       0.0|
|(308,[0,1,2,6,7,8...|    0|[55.1033356722794...|[0.55103335672279...|       0.0|
|(308,[0,1,2,6,7,8...|    1|[54.2268769488640...|[0.54226876948864...|       0.0|
|(308,[0,1,2,6,7,8...|    1|[51.2890848104632...|[0.51289084810463...|       0.0|
|(308,[0,1,2,6,7,8...|    1|[52.1067965184036...|[0.52106796518403...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [26]:
# http://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html?highlight=classificationevaluator#pyspark.ml.evaluation.MulticlassClassificationEvaluator.metricName
evaluator = MulticlassClassificationEvaluator(metricName='accuracy')
rf_accuracy = evaluator.evaluate(prediction)
rf_accuracy

0.7547786720321932

In [27]:
rf_auc = BinaryClassificationEvaluator(
    labelCol='label').evaluate(prediction)
rf_auc 

0.7226826849686644

# **TEST HISTORY**

In [28]:
testall = spark.read.csv('/content/drive/MyDrive/project/data/all_features_test_OneHot.csv', header=True, inferSchema=True)
testall.show(1)

+---+-----+-----------+--------+--------------------+---------------------+------------------+---------------------------------+-------------------------+------------------------+-----------+---------------+---------------+------------------+--------------------+--------------------+---------------------+-----------------------+-----------------------+---------------------+-----------------------+-----------------------+---------------------+-----------------------+-----------------------+----------------------+------------------------+------------------------+-------------------+---------------------+---------------------+----------------------+------------------------+------------------------+----------------------+------------------------+------------------------+----------------------+------------------------+------------------------+-----------------------+-------------------------+-------------------------+----------------+------------------+------------------+-------------------

In [30]:
feature_cols = testall.columns[4:]
print(feature_cols)
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
assembler
testall = assembler.transform(testall)

['never_bought_company', 'never_bought_category', 'never_bought_brand', 'has_bought_brand_company_category', 'has_bought_brand_category', 'has_bought_brand_company', 'offer_value', 'total_spend_all', 'total_spend_ccb', 'has_bought_company', 'has_bought_company_q', 'has_bought_company_a', 'has_bought_company_30', 'has_bought_company_q_30', 'has_bought_company_a_30', 'has_bought_company_60', 'has_bought_company_q_60', 'has_bought_company_a_60', 'has_bought_company_90', 'has_bought_company_q_90', 'has_bought_company_a_90', 'has_bought_company_180', 'has_bought_company_q_180', 'has_bought_company_a_180', 'has_bought_category', 'has_bought_category_q', 'has_bought_category_a', 'has_bought_category_30', 'has_bought_category_q_30', 'has_bought_category_a_30', 'has_bought_category_60', 'has_bought_category_q_60', 'has_bought_category_a_60', 'has_bought_category_90', 'has_bought_category_q_90', 'has_bought_category_a_90', 'has_bought_category_180', 'has_bought_category_q_180', 'has_bought_categ

In [31]:
testsetall = testall.select(['features'])

In [32]:
predict_test = model.transform(testsetall)
predict_test.show(5)

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(308,[1,5,6,7,8,9...|[71.8554986971595...|[0.71855498697159...|       0.0|
|(308,[3,4,5,6,7,8...|[64.9574046790083...|[0.64957404679008...|       0.0|
|(308,[0,2,6,7,8,2...|[80.6320141923426...|[0.80632014192342...|       0.0|
|(308,[2,6,7,8,9,1...|[75.1588703048051...|[0.75158870304805...|       0.0|
|(308,[2,6,7,8,9,1...|[67.6930447963485...|[0.67693044796348...|       0.0|
+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [33]:
predict_test.toPandas().to_csv('/content/drive/MyDrive/project/data/PREDICT_TEST_ALL.csv')

/content/spark-3.1.2-bin-hadoop2.7/python/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


# **ROC CURVE**

In [1]:
import numpy as np
data['prob'] = data['prob'].map(lambda x: list(x))
data['prediction'] = data['prediction'].map(lambda x: np.eye(2)[int(x)])

NameError: ignored

In [ ]:
y_pred = np.array(pred_pd['probability'].tolist())
y_true = np.array(pred_pd['encoded_label'].tolist())

In [ ]:
from sklearn.metrics import auc, roc_curve

fpr, tpr, threshold = roc_curve(y_score=y_pred[:,0], y_true=y_true[:,0])
auc = auc(fpr, tpr)

print('AUC: {:.3f}'.format(auc))

In [ ]:
plt.figure()
plt.plot([0,1], [0,1], 'k--', color='orange')
plt.plot(fpr, tpr, label='auc = {:.3f}'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='lower right')
plt.grid()
plt.show()

# **Grid Search**